# Парсим HeadHunter (hh.ru)

### Импортируем все самое нужное

In [1]:
import requests
import sys, os
import json
import re
import pandas as pd
import time

### У hh.ru есть официальное API 
Парсить сам сайт просто GET-запросами они не дают, надо либо вести себя как браузер (см. selenium) либо воспользоваться API. Мы выбираем второй вариант. Посмотрим, какие у них есть специальности

In [2]:
url = 'https://api.hh.ru/specializations'

In [3]:
page = requests.get(url).content
page[:200]

b'[{"specializations":[{"id":"24.492","name":"\xd0\x9c\xd0\xb0\xd1\x81\xd1\x81\xd0\xb0\xd0\xb6\xd0\xb8\xd1\x81\xd1\x82"},{"id":"24.493","name":"\xd0\x9f\xd0\xb0\xd1\x80\xd0\xb8\xd0\xba\xd0\xbc\xd0\xb0\xd1\x85\xd0\xb5\xd1\x80"},{"id":"24.624","name":"\xd0\x9d\xd0\xbe\xd0\xb3\xd1\x82\xd0\xb5\xd0\xb2\xd0\xbe\xd0\xb9 \xd1\x81\xd0\xb5\xd1\x80\xd0\xb2\xd0\xb8\xd1\x81"},{"id":"24.377","name":"\xd0\x9a\xd0\xbe\xd1\x81\xd0\xbc\xd0\xb5\xd1'

Все данные в побайтовом виде - надо привести их к юникоду (декодировать). Делаем это:

In [4]:
page = page.decode(encoding='utf-8').lower()
page[:200]

'[{"specializations":[{"id":"24.492","name":"массажист"},{"id":"24.493","name":"парикмахер"},{"id":"24.624","name":"ногтевой сервис"},{"id":"24.377","name":"косметология"},{"id":"24.378","name":"тренер'

Пытаемся найти среди этих специальностей программирование:

In [5]:
pos = page.find('программирование')
page[pos-40:pos+40]

'ержка, helpdesk"},{"id":"1.221","name":"программирование, разработка"},{"id":"1.'

Значит, нам нужна специальность 1.221

In [6]:
SPECID = '1.221'
AREA = 1  # == Moscow

### Спарсим какую-нибудь одну вакансию
На одной странице мы будем выводить по одной вакансии (так удобней всего с точки зрения кода). Выведем первую страницу (т.е. страницу №0):

In [7]:
urltemplate = 'https://api.hh.ru/vacancies?specialization={}&area={}&only_with_salary=true&per_page=1&page={}'
url = urltemplate.format(SPECID, AREA, 0)

In [8]:
content = requests.get(url).content.decode(encoding='utf-8').lower()
content

'{"clusters":null,"items":[{"salary":{"to":175000,"from":120000,"currency":"rur"},"snippet":{"requirement":"php - сильные и слабые стороны языка. иметь опыт проектирования реляционных баз данных (postgresql, mysql), опыт работы с большими объемами данных. ","responsibility":"разрабатывать приложение по приему и обработке онлайн платежей в сфере эквайринговых cnp решений и мобильной коммерции. активно участвовать во всём..."},"archived":false,"premium":false,"name":"senior php developer","area":{"url":"https://api.hh.ru/areas/1","id":"1","name":"москва"},"url":"https://api.hh.ru/vacancies/18322907?host=hh.ru","created_at":"2016-11-05t22:52:42+0300","apply_alternate_url":"https://hh.ru/applicant/vacancy_response?vacancyid=18322907","relations":[],"employer":{"logo_urls":{"90":"https://hhcdn.ru/employer-logo/1777010.jpeg","original":"https://hhcdn.ru/employer-logo-original/333665.jpg","240":"https://hhcdn.ru/employer-logo/1777011.jpeg"},"vacancies_url":"https://api.hh.ru/vacancies?employe

### Сделаем из этого JSON

In [9]:
content = json.loads(content)

In [10]:
content['items']

[{'address': {'building': '2с4',
   'city': 'москва',
   'description': None,
   'id': '500788',
   'lat': 55.716529,
   'lng': 37.64586,
   'metro': {'lat': 55.729741,
    'line_id': '2',
    'line_name': 'замоскворецкая',
    'lng': 37.638693,
    'station_id': '2.101',
    'station_name': 'павелецкая'},
   'metro_stations': [{'lat': 55.729741,
     'line_id': '2',
     'line_name': 'замоскворецкая',
     'lng': 37.638693,
     'station_id': '2.101',
     'station_name': 'павелецкая'}],
   'raw': None,
   'street': 'павелецкая набережная'},
  'alternate_url': 'https://hh.ru/vacancy/18322907',
  'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyid=18322907',
  'archived': False,
  'area': {'id': '1', 'name': 'москва', 'url': 'https://api.hh.ru/areas/1'},
  'created_at': '2016-11-05t22:52:42+0300',
  'department': None,
  'employer': {'alternate_url': 'https://hh.ru/employer/802558',
   'id': '802558',
   'logo_urls': {'240': 'https://hhcdn.ru/employer-logo/177701

In [11]:
smallurl = content['items'][0]['url']
smallurl

'https://api.hh.ru/vacancies/18322907?host=hh.ru'

### Скачаем саму вакансию
Теперь, когда мы получили URL вакансии, мы можем посмотреть на нее более детально

In [12]:
job = requests.get(smallurl).content.decode(encoding='utf-8').lower()
job[:200]

'{"alternate_url":"https://hh.ru/vacancy/18322907","code":null,"premium":false,"description":"<p><strong>ecommpay it</strong> - европейская высокотехнологичная компания, предлагающая комплексные решени'

In [13]:
job = json.loads(job)

In [14]:
job

{'accept_handicapped': False,
 'address': {'building': '2с4',
  'city': 'москва',
  'description': None,
  'lat': 55.716529,
  'lng': 37.64586,
  'metro': {'lat': 55.729741,
   'line_id': '2',
   'line_name': 'замоскворецкая',
   'lng': 37.638693,
   'station_id': '2.101',
   'station_name': 'павелецкая'},
  'metro_stations': [{'lat': 55.729741,
    'line_id': '2',
    'line_name': 'замоскворецкая',
    'lng': 37.638693,
    'station_id': '2.101',
    'station_name': 'павелецкая'}],
  'raw': None,
  'street': 'павелецкая набережная'},
 'allow_messages': True,
 'alternate_url': 'https://hh.ru/vacancy/18322907',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyid=18322907',
 'archived': False,
 'area': {'id': '1',
  'name': 'москва',
  'url': 'https://api.hh.ru/areas/1?host=hh.ru'},
 'billing_type': {'id': 'standard_plus', 'name': 'стандарт+'},
 'branded_description': None,
 'code': None,
 'contacts': None,
 'created_at': '2016-11-05t22:52:42+0300',
 'department':

### Отсеиваем неподходящие вакансии

In [15]:
currency = job['salary']['currency']
salaryMin = job['salary']['from']
salaryMax = job['salary']['to']
employment = job['employment']['id']

Не будем принимать вакансии, в которых цена указана не в рублях (обычно это значит, что работодатель не из России). Также не будем принимать вакансии с неполной занятостью (они будут портить статистику) и вакансии, в которых не указан минимальный размер зарплаты (вакансии вида "до 40000 руб."):

In [16]:
if salaryMin == 'none' or currency != 'rur' or employment != 'full':
    print('BAD VACANCY!!')

В качестве зарплаты возьмем среднюю между максимальной и минимальной зарплатой (середину вилки). Если указана только минимальная, будем использвать ее:

In [17]:
if salaryMax == None:
    salaryMax = salaryMin
salary = 0.5 * (salaryMin + salaryMax)
int(salary)

147500

### Начнем собирать словарик из данных
Позже (в том ноутбуке, где мы будем анализировать данные) станет понятно, почему я начинаю названия столбцов с символа подчеркивания

In [18]:
data_row = {}

In [19]:
data_row['_salary'] = int(salary)

In [20]:
desc = job['description']
desc

'<p><strong>ecommpay it</strong> - европейская высокотехнологичная компания, предлагающая комплексные решения по приему и обработке онлайн платежей в сфере эквайринговых cnp решений и мобильной коммерции. <strong>ecommpay it </strong>входит в группу компаний <strong>ecommpay</strong>, которая является провайдером инновационных и уникальных платежных решений для широкого круга проектов э-коммерции во всех странах мира.</p> <p>компания успешно продвигает инновационные финансовые решения в области электронных платежей и уже реализовала более 30-ти крупных e- и m-commerce проектов. благодаря команде ит-профессионалов платежной индустрии, качественному менеджменту и собственному отлаженному производству программного обеспечения.</p> <p><strong>ecommpay it</strong> помогает своим клиентам запускать новые эффективные продукты и услуги.</p> <p> </p> <p>мы ищем в команду опытных разработчиков, увлеченных своим делом, которые готовы участвовать в разработке большого проекта.</p> <p> </p> <p><str

Уберем из описания все HTML-тэги:

In [21]:
desc = re.sub('<[^<]+?>', '', desc)
desc

'ecommpay it - европейская высокотехнологичная компания, предлагающая комплексные решения по приему и обработке онлайн платежей в сфере эквайринговых cnp решений и мобильной коммерции. ecommpay it входит в группу компаний ecommpay, которая является провайдером инновационных и уникальных платежных решений для широкого круга проектов э-коммерции во всех странах мира. компания успешно продвигает инновационные финансовые решения в области электронных платежей и уже реализовала более 30-ти крупных e- и m-commerce проектов. благодаря команде ит-профессионалов платежной индустрии, качественному менеджменту и собственному отлаженному производству программного обеспечения. ecommpay it помогает своим клиентам запускать новые эффективные продукты и услуги.   мы ищем в команду опытных разработчиков, увлеченных своим делом, которые готовы участвовать в разработке большого проекта.   чем придётся заниматься:  разрабатывать приложение по приему и обработке онлайн платежей в сфере эквайринговых cnp ре

### Скоро мы сможем смотреть, что же нужно работодателю

In [22]:
'php' in desc

True

In [23]:
'ruby' in desc

False

In [24]:
data_row['_description'] = desc
data_row['_id'] = job['id']
data_row['_name'] = job['name']

In [25]:
data_row

{'_description': 'ecommpay it - европейская высокотехнологичная компания, предлагающая комплексные решения по приему и обработке онлайн платежей в сфере эквайринговых cnp решений и мобильной коммерции. ecommpay it входит в группу компаний ecommpay, которая является провайдером инновационных и уникальных платежных решений для широкого круга проектов э-коммерции во всех странах мира. компания успешно продвигает инновационные финансовые решения в области электронных платежей и уже реализовала более 30-ти крупных e- и m-commerce проектов. благодаря команде ит-профессионалов платежной индустрии, качественному менеджменту и собственному отлаженному производству программного обеспечения. ecommpay it помогает своим клиентам запускать новые эффективные продукты и услуги.   мы ищем в команду опытных разработчиков, увлеченных своим делом, которые готовы участвовать в разработке большого проекта.   чем придётся заниматься:  разрабатывать приложение по приему и обработке онлайн платежей в сфере экв

### Словарик получен, делаем из него DataFrame

In [26]:
pd.DataFrame(data_row, columns=['_id', '_name', '_description', '_salary'], index=[0])

,_id,_name,_description,_salary
0,18322907,senior php developer,ecommpay it - европейская высокотехнологичная ...,147500


### Повторяем то же самое, но для 2000 вакансий
Создаем список словариков rows, потом сделаем из них один большой DataFrame 

In [76]:
NPAGES = 2000
rows = []
j = 0
for i in range(NPAGES):
    if i % 50 == 0:
        print('{0:5.1%} completed'.format(i/NPAGES))
    url = urltemplate.format(SPECID, AREA, i)
    time.sleep(0.05)
    content = requests.get(url).content.decode(encoding='utf-8').lower()
    content = json.loads(content)
    smallurl = content['items'][0]['url']
    time.sleep(0.05)
    job = requests.get(smallurl).content.decode(encoding='utf-8').lower()
    job = json.loads(job)
    currency = job['salary']['currency']
    salaryF = job['salary']['from']
    salaryT = job['salary']['to']
    employment = job['employment']['id']
    if salaryF == None or currency != 'rur' or employment != 'full':
        continue
    if salaryT == None:
        salaryT = salaryF
    salary = 0.5 * (salaryT + salaryF)
    data_row = {}
    data_row['_salary'] = int(salary)
    desc = re.sub('<[^<]+?>', '', job['description'])
    data_row['_description'] = desc
    data_row['_id'] = job['id']
    data_row['_name'] = job['name']
    rows += [ pd.DataFrame(data_row, columns=['_id', '_name', '_description', '_salary'], index=[j]) ]
    j += 1

### Собираем DataFrame, пишем его в файл

In [79]:
df = pd.concat(rows, axis=0)
df.head(5)

,_id,_name,_description,_salary
0,18322907,senior php developer,ecommpay it - европейская высокотехнологичная ...,147500
1,18809284,senior frontend developer,русскоязычная команда программистов компании c...,120000
2,18686089,"старший разработчик c#, asp.net mvc",&quot;додо пицца&quot; ищет талантливых разраб...,185000
3,18714668,web-программист,"требования: знание php, javascript, html, c...",45000
4,18774080,разработчик мобильных приложений под android,английский стартап - мобильное приложение для ...,155000


In [80]:
df.to_csv('rawdata.csv', index=False)